# Analyse par Régime : Adaptive vs Fixed

**Objectif** : Comprendre où et pourquoi l'optimisation Adaptive sous-performe vs Fixed.

**Méthodologie** :
1. Charger les résultats Walk-Forward existants
2. Découper les performances par régime (Bull 2020, Bear 2022, Recovery 2023, Bull 2024)
3. Générer heatmaps Δ(Adaptive - Fixed) par régime × profil
4. Identifier les régimes/profils où Adaptive perd

**Données** : Résultats de `optimize_multi_envelope.ipynb` (déjà calculés)

In [ ]:
# Imports
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

print("✅ Imports OK")

## 1. Chargement des Résultats Existants

In [ ]:
# Trouver le fichier de résultats le plus récent
results_dir = Path('.')
wf_files = sorted(results_dir.glob('wf_results_detailed_*.csv'), reverse=True)

if len(wf_files) == 0:
    print("❌ ERREUR: Aucun fichier wf_results_detailed_*.csv trouvé")
    print("   Vous devez d'abord exécuter optimize_multi_envelope.ipynb en MODE PRODUCTION")
    raise FileNotFoundError("No optimization results found")

latest_file = wf_files[0]
print(f"📂 Fichier de résultats : {latest_file.name}")
print(f"   Date : {latest_file.stat().st_mtime}")

# Charger les résultats
df_wf = pd.read_csv(latest_file)

print(f"\n✅ {len(df_wf)} résultats chargés")
print(f"   Colonnes : {list(df_wf.columns)}")
print(f"\nAperçu :")
df_wf.head()

## 2. Définition des Régimes

Périodes de marché basées sur les cycles BTC :

In [ ]:
# Définition des régimes (dates exactes des cycles Bitcoin)
REGIMES = {
    'bull_2020': {
        'start': '2020-03-13',
        'end': '2021-11-10',
        'label': 'Bull 2020-2021',
        'desc': 'COVID bottom $3,850 → ATH $69,000'
    },
    'bear_2022': {
        'start': '2021-11-10',
        'end': '2022-11-21',
        'label': 'Bear 2021-2022',
        'desc': 'ATH $69k → FTX crash $15,479'
    },
    'recovery_2023': {
        'start': '2022-11-22',
        'end': '2023-12-31',
        'label': 'Recovery 2023',
        'desc': 'FTX bottom $16k → Recovery $42k'
    },
    'bull_2024': {
        'start': '2024-01-01',
        'end': '2024-10-03',
        'label': 'Bull 2024',
        'desc': 'ETF approval → ATH $73,800'
    }
}

print("Régimes définis :")
for regime_id, info in REGIMES.items():
    print(f"  {info['label']:20s} : {info['start']} → {info['end']}")
    print(f"  {'':20s}   {info['desc']}")

## 3. Analyse Exploratoire des Résultats

Comprendre la structure des données avant analyse par régime.

In [ ]:
# Vérifier la structure des résultats
print("=" * 80)
print("STRUCTURE DES RÉSULTATS")
print("=" * 80)

# Colonnes disponibles
print(f"\nColonnes disponibles : {len(df_wf.columns)}")
for col in df_wf.columns:
    print(f"  - {col}")

# Profils testés
if 'profile' in df_wf.columns:
    print(f"\nProfils : {df_wf['profile'].unique().tolist()}")
    print(f"  Counts : {df_wf['profile'].value_counts().to_dict()}")

# Adaptive vs Fixed
if 'adaptive' in df_wf.columns:
    print(f"\nAdaptive :")
    print(f"  True  : {(df_wf['adaptive'] == True).sum()} configs")
    print(f"  False : {(df_wf['adaptive'] == False).sum()} configs")

# Folds
if 'fold' in df_wf.columns:
    print(f"\nFolds : {sorted(df_wf['fold'].unique())}")

# Statistiques Sharpe
if 'sharpe_test' in df_wf.columns:
    print(f"\nSharpe Test :")
    print(f"  Mean   : {df_wf['sharpe_test'].mean():.2f}")
    print(f"  Median : {df_wf['sharpe_test'].median():.2f}")
    print(f"  Std    : {df_wf['sharpe_test'].std():.2f}")
    print(f"  Min    : {df_wf['sharpe_test'].min():.2f}")
    print(f"  Max    : {df_wf['sharpe_test'].max():.2f}")

print("\n" + "="*80)

## 4. Comparaison Adaptive vs Fixed (Global)

Avant d'analyser par régime, regardons la différence globale.

In [ ]:
# Comparaison globale Adaptive vs Fixed
if 'adaptive' in df_wf.columns and 'sharpe_test' in df_wf.columns:
    
    df_fixed = df_wf[df_wf['adaptive'] == False]
    df_adaptive = df_wf[df_wf['adaptive'] == True]
    
    print("=" * 80)
    print("COMPARAISON GLOBALE : ADAPTIVE vs FIXED")
    print("=" * 80)
    
    metrics = ['sharpe_test', 'sharpe_train', 'score_test', 'n_trades_test']
    
    comparison = pd.DataFrame({
        'Metric': [],
        'Fixed (Mean)': [],
        'Adaptive (Mean)': [],
        'Δ (Adaptive - Fixed)': [],
        'Δ %': []
    })
    
    for metric in metrics:
        if metric in df_wf.columns:
            fixed_mean = df_fixed[metric].mean()
            adaptive_mean = df_adaptive[metric].mean()
            delta = adaptive_mean - fixed_mean
            delta_pct = (delta / fixed_mean * 100) if fixed_mean != 0 else 0
            
            comparison = pd.concat([comparison, pd.DataFrame({
                'Metric': [metric],
                'Fixed (Mean)': [f"{fixed_mean:.2f}"],
                'Adaptive (Mean)': [f"{adaptive_mean:.2f}"],
                'Δ (Adaptive - Fixed)': [f"{delta:+.2f}"],
                'Δ %': [f"{delta_pct:+.1f}%"]
            })], ignore_index=True)
    
    print("\n", comparison.to_string(index=False))
    
    # Verdict
    sharpe_delta = df_adaptive['sharpe_test'].mean() - df_fixed['sharpe_test'].mean()
    
    print("\n" + "="*80)
    if sharpe_delta > 0:
        print(f"✅ Adaptive GAGNE : Sharpe +{sharpe_delta:.2f} ({sharpe_delta/df_fixed['sharpe_test'].mean()*100:+.1f}%)")
    else:
        print(f"❌ Adaptive PERD : Sharpe {sharpe_delta:.2f} ({sharpe_delta/df_fixed['sharpe_test'].mean()*100:+.1f}%)")
    print("="*80)
    
else:
    print("⚠️  Colonnes 'adaptive' ou 'sharpe_test' manquantes - skip comparaison")

## 5. Analyse par Régime

**IMPORTANT** : Cette analyse nécessite les données `df_days` individuelles par backtest, qui ne sont pas stockées dans `wf_results_detailed.csv`.

**Deux options** :
1. **Option A (Lecture directe)** : Si les pickles/parquets de résultats sont sauvegardés
2. **Option B (Re-calcul)** : Re-lancer les backtests des meilleures configs et analyser df_days

Pour l'instant, analysons ce qu'on peut avec les données agrégées disponibles.

In [ ]:
# Vérifier si on a les données temporelles pour découper par régime
temporal_cols = ['start_date', 'end_date', 'test_start', 'test_end', 'train_start', 'train_end']
available_temporal = [col for col in temporal_cols if col in df_wf.columns]

print(f"Colonnes temporelles disponibles : {available_temporal}")

if len(available_temporal) > 0:
    print("\n✅ Analyse par régime possible via mapping folds → régimes")
    print("   Étape suivante : Mapper chaque fold à son régime dominant")
else:
    print("\n⚠️  Pas de colonnes temporelles - analyse par régime limitée")
    print("   Solution : Re-lancer backtests des meilleures configs avec sauvegarde df_days")

## 6. Analyse par Profil

En attendant l'analyse temporelle complète, analysons par profil.

In [ ]:
# Analyse Adaptive vs Fixed par profil
if 'profile' in df_wf.columns and 'adaptive' in df_wf.columns:
    
    print("=" * 80)
    print("COMPARAISON PAR PROFIL : ADAPTIVE vs FIXED")
    print("=" * 80)
    
    profiles = df_wf['profile'].unique()
    
    results = []
    
    for profile in profiles:
        df_profile = df_wf[df_wf['profile'] == profile]
        
        df_fixed_prof = df_profile[df_profile['adaptive'] == False]
        df_adaptive_prof = df_profile[df_profile['adaptive'] == True]
        
        if len(df_fixed_prof) > 0 and len(df_adaptive_prof) > 0:
            sharpe_fixed = df_fixed_prof['sharpe_test'].mean()
            sharpe_adaptive = df_adaptive_prof['sharpe_test'].mean()
            delta_sharpe = sharpe_adaptive - sharpe_fixed
            delta_pct = (delta_sharpe / sharpe_fixed * 100) if sharpe_fixed != 0 else 0
            
            n_trades_fixed = df_fixed_prof['n_trades_test'].mean()
            n_trades_adaptive = df_adaptive_prof['n_trades_test'].mean()
            
            results.append({
                'Profil': profile,
                'Sharpe Fixed': f"{sharpe_fixed:.2f}",
                'Sharpe Adaptive': f"{sharpe_adaptive:.2f}",
                'Δ Sharpe': f"{delta_sharpe:+.2f}",
                'Δ %': f"{delta_pct:+.1f}%",
                'Trades Fixed': f"{n_trades_fixed:.0f}",
                'Trades Adaptive': f"{n_trades_adaptive:.0f}",
                'Verdict': '✅ Adaptive' if delta_sharpe > 0 else '❌ Fixed'
            })
    
    df_results = pd.DataFrame(results)
    print("\n", df_results.to_string(index=False))
    
    print("\n" + "="*80)
    print("RÉSUMÉ")
    print("="*80)
    
    adaptive_wins = sum([1 for r in results if '✅' in r['Verdict']])
    print(f"  Adaptive gagne sur {adaptive_wins}/{len(results)} profils")
    
    if adaptive_wins < len(results) / 2:
        print(f"\n❌ Conclusion : Adaptive perd sur la majorité des profils")
        print(f"   → L'approche Fixed reste meilleure globalement")
    else:
        print(f"\n✅ Conclusion : Adaptive gagne sur la majorité des profils")
    
    print("="*80)
    
else:
    print("⚠️  Colonnes 'profile' ou 'adaptive' manquantes")

## 7. Visualisation : Heatmap Δ(Adaptive - Fixed) par Profil

Heatmap montrant où Adaptive gagne (vert) ou perd (rouge) vs Fixed.

In [ ]:
# Heatmap Δ Sharpe par profil
if 'profile' in df_wf.columns and 'adaptive' in df_wf.columns:
    
    # Préparer données pour heatmap
    profiles = sorted(df_wf['profile'].unique())
    metrics = ['sharpe_test', 'score_test', 'n_trades_test']
    
    delta_data = []
    
    for profile in profiles:
        row = {'Profil': profile}
        
        df_profile = df_wf[df_wf['profile'] == profile]
        df_fixed = df_profile[df_profile['adaptive'] == False]
        df_adaptive = df_profile[df_profile['adaptive'] == True]
        
        for metric in metrics:
            if metric in df_wf.columns:
                fixed_val = df_fixed[metric].mean()
                adaptive_val = df_adaptive[metric].mean()
                delta = adaptive_val - fixed_val
                row[metric] = delta
        
        delta_data.append(row)
    
    df_delta = pd.DataFrame(delta_data).set_index('Profil')
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(10, 6))
    
    sns.heatmap(
        df_delta,
        annot=True,
        fmt=".2f",
        cmap='RdYlGn',
        center=0,
        cbar_kws={'label': 'Δ (Adaptive - Fixed)'},
        linewidths=0.5,
        ax=ax
    )
    
    ax.set_title('Heatmap Δ(Adaptive - Fixed) par Profil', fontsize=14, fontweight='bold')
    ax.set_xlabel('Métrique', fontsize=12)
    ax.set_ylabel('Profil', fontsize=12)
    
    plt.tight_layout()
    plt.savefig('heatmap_adaptive_vs_fixed_by_profile.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✅ Heatmap sauvegardée : heatmap_adaptive_vs_fixed_by_profile.png")
    
else:
    print("⚠️  Données insuffisantes pour heatmap")

## 8. Diagnostic et Recommandations

Synthèse des findings et prochaines étapes.

In [ ]:
print("=" * 80)
print("DIAGNOSTIC FINAL")
print("=" * 80)

findings = []

# Finding 1: Adaptive vs Fixed global
if 'adaptive' in df_wf.columns:
    df_fixed = df_wf[df_wf['adaptive'] == False]
    df_adaptive = df_wf[df_wf['adaptive'] == True]
    
    sharpe_delta = df_adaptive['sharpe_test'].mean() - df_fixed['sharpe_test'].mean()
    
    if sharpe_delta < -0.1:
        findings.append({
            'Finding': 'Adaptive sous-performe globalement',
            'Impact': f'Sharpe {sharpe_delta:.2f}',
            'Recommendation': 'Utiliser Fixed params (pas de régime adaptation)'
        })
    elif sharpe_delta > 0.1:
        findings.append({
            'Finding': 'Adaptive sur-performe globalement',
            'Impact': f'Sharpe +{sharpe_delta:.2f}',
            'Recommendation': 'Utiliser Adaptive params (avec régimes)'
        })
    else:
        findings.append({
            'Finding': 'Adaptive ≈ Fixed (performance similaire)',
            'Impact': f'Sharpe Δ {sharpe_delta:.2f}',
            'Recommendation': 'Utiliser Fixed (plus simple, même résultat)'
        })

# Finding 2: Nombre de trades
if 'n_trades_test' in df_wf.columns:
    avg_trades = df_wf['n_trades_test'].mean()
    
    if avg_trades < 50:
        findings.append({
            'Finding': 'Échantillon de trades insuffisant',
            'Impact': f'Moyenne {avg_trades:.0f} trades/config',
            'Recommendation': 'Élargir fenêtres envelopes ou réduire MA'
        })

# Finding 3: Profils problématiques
if 'profile' in df_wf.columns:
    for profile in df_wf['profile'].unique():
        df_prof = df_wf[df_wf['profile'] == profile]
        
        if 'n_trades_test' in df_prof.columns:
            if df_prof['n_trades_test'].mean() < 30:
                findings.append({
                    'Finding': f'Profil "{profile}" : trop peu de trades',
                    'Impact': f'{df_prof["n_trades_test"].mean():.0f} trades en moyenne',
                    'Recommendation': f'Fusionner avec autre profil ou ajuster params'
                })

# Afficher findings
if len(findings) > 0:
    df_findings = pd.DataFrame(findings)
    print("\n", df_findings.to_string(index=False))
else:
    print("\n✅ Aucun problème majeur détecté")

print("\n" + "="*80)
print("PROCHAINES ÉTAPES")
print("="*80)
print("""
1. **Étape 1b** : Calculer volatilité réalisée par pair → mapping nb envelopes
   
2. **Étape 2a** : Re-optimisation avec multiplicateurs + nb envelopes variable
   - Grilles réduites (multiplicateurs au lieu de valeurs absolues)
   - 3 env pour majors/low, 4 env pour mid-cap/volatiles
   - Scoring corrigé (exclusions au lieu de pénalités -500)
   
3. **Étape 2b** : Analyse 3env vs 4env post-optimisation
   - Valider hypothèse "4env meilleur sur volatiles"
   
4. **Étape 3** : Gate v2 hiérarchique pour décision finale
   - Tier 1 (HARD) + Tier 2 (SOFT) moins restrictifs
   - Recommandation Global vs Profils
""")
print("="*80)